In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:13 http://security.ubuntu.com/ubuntu focal-securi

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-01-28 04:03:20--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.76MB/s    in 0.2s    

2023-01-28 04:03:21 (5.76 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [5]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz" 
spark.sparkContext.addFile(url)
digital_music_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz"), sep='\t', header=True, inferSchema=True, timestampFormat="yyyy/MM/dd")

# Show DataFrame
digital_music_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10293599|R14LVB34Z2Z53I|B000YMOQZY|     262935067|Knockin' On Heave...|Digital_Music_Pur...|          5|            0|          0|   N|                Y|           favorites|Great  rendition....| 2015-08-31|
|         US|    6087195|R2M0R9CWPC82LT|B00ISAEC80|     452318038|            Flawless|Digital_Music_Pur

In [6]:
# Get the number of rows in the DataFrame.

digital_music_df.count()

1688884

# Transform the Data

In [7]:

digital_music_df.printSchema()


root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



## Create the "review_id_table".

In [8]:
from pyspark.sql.functions import to_date

# Create the "review_id_df" DataFrame with the appropriate columns and data types.

review_id_df = digital_music_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show()
     

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R14LVB34Z2Z53I|   10293599|B000YMOQZY|     262935067| 2015-08-31|
|R2M0R9CWPC82LT|    6087195|B00ISAEC80|     452318038| 2015-08-31|
| RQNQI2R3BM52A|   33717063|B00VBB6AVU|     675969129| 2015-08-31|
| RTUIX1135HUAB|   14948020|B008A5POJO|     282925893| 2015-08-31|
| RE3QSAT8TL010|   21694522|B014CW2RN4|     109077793| 2015-08-31|
|R14TN65IY0NUOD|   14948020|B00DRA3EOY|     959245080| 2015-08-31|
|R3LIF8MCNH1ODF|   20031966|B009G3U0PW|     762436916| 2015-08-31|
|R33X0DGA4PPQ3L|   12223745|B013XBYQOS|     137980344| 2015-08-31|
|R2DJJP7DQRT1ZW|   30231876|B003L5H25W|     165995375| 2015-08-31|
|R2A8XDXW5XOT4H|   51714365|B00136NVH4|     227508666| 2015-08-31|
|R3MJK3L7ER61TT|   37216001|B0030AKPQ6|     164096943| 2015-08-31|
|R38Z7XMCVDR0R5|   49207156|B014EQE0V0|     714368268| 2015-08

In [11]:
# Check datatype

review_id_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [10]:
# Convert "review date" from string to date

review_id_df = digital_music_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R14LVB34Z2Z53I|   10293599|B000YMOQZY|     262935067| 2015-08-31|
|R2M0R9CWPC82LT|    6087195|B00ISAEC80|     452318038| 2015-08-31|
| RQNQI2R3BM52A|   33717063|B00VBB6AVU|     675969129| 2015-08-31|
| RTUIX1135HUAB|   14948020|B008A5POJO|     282925893| 2015-08-31|
| RE3QSAT8TL010|   21694522|B014CW2RN4|     109077793| 2015-08-31|
|R14TN65IY0NUOD|   14948020|B00DRA3EOY|     959245080| 2015-08-31|
|R3LIF8MCNH1ODF|   20031966|B009G3U0PW|     762436916| 2015-08-31|
|R33X0DGA4PPQ3L|   12223745|B013XBYQOS|     137980344| 2015-08-31|
|R2DJJP7DQRT1ZW|   30231876|B003L5H25W|     165995375| 2015-08-31|
|R2A8XDXW5XOT4H|   51714365|B00136NVH4|     227508666| 2015-08-31|
|R3MJK3L7ER61TT|   37216001|B0030AKPQ6|     164096943| 2015-08-31|
|R38Z7XMCVDR0R5|   49207156|B014EQE0V0|     714368268| 2015-08

In [12]:
# Re-Check the datatype

review_id_df.dtypes


[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

## Create the "products" Table

In [13]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 

products_df = digital_music_df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B014PO1708|       See You Again|
|B001APIS9Y|What Happened to ...|
|B005XT4CAW|I Can Dream About...|
|B006WH9FPQ|  If You Want To Fly|
|B0026G2MG6|Ghetto Fabolous [...|
|B0096RUQZW|       Call Me Maybe|
|B003YZGVFQ|Maxwell's silver ...|
|B00138B7N2|He Touched Me (Al...|
|B00OLF0U3M|Beyond The Lights...|
|B007D89P76|A Hampshire Suite...|
|B0011U5E1S|             Nirvana|
|B00JWFMVNS|Sousa (Red Classics)|
|B00120EDBY|Talkin' To You Ey...|
|B00IWI3HNO|          Sweet Life|
|B00YIB4KJY|                Fall|
|B0011ZVFIO|Carmel City (LP V...|
|B00C32RB4Q|         Lola Montez|
|B001NTZ4NA|Thug Lovin' (Albu...|
|B001BHJZN4|      Eye In The Sky|
|B005SR8QO2|      My Kinda Party|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [14]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 

customers_df = digital_music_df.groupby("customer_id")\
              .agg({"customer_id": "count"})\
              .withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()
     

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   39290568|             1|
|   23601866|             1|
|   19004376|             1|
|   16320823|             1|
|   13973119|             1|
|    1155154|             1|
|   24317024|             2|
|   15125275|             1|
|   35781732|             2|
|   40985731|            12|
|   19522542|             1|
|   50243180|             1|
|    2702985|             1|
|   41857916|             1|
|    1943933|             2|
|   42050930|             5|
|    4867895|             1|
|   46317087|             3|
|   52021773|             2|
|   15504622|             9|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [15]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 

from pyspark.sql.functions import col

vine_df = digital_music_df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])
vine_df.show()
     


+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R14LVB34Z2Z53I|          5|            0|          0|   N|
|R2M0R9CWPC82LT|          5|            0|          0|   N|
| RQNQI2R3BM52A|          4|            0|          0|   N|
| RTUIX1135HUAB|          5|            0|          0|   N|
| RE3QSAT8TL010|          4|            0|          0|   N|
|R14TN65IY0NUOD|          5|            0|          0|   N|
|R3LIF8MCNH1ODF|          5|            0|          0|   N|
|R33X0DGA4PPQ3L|          5|            3|          3|   N|
|R2DJJP7DQRT1ZW|          5|            0|          1|   N|
|R2A8XDXW5XOT4H|          5|            0|          0|   N|
|R3MJK3L7ER61TT|          5|            0|          0|   N|
|R38Z7XMCVDR0R5|          5|            1|          1|   N|
|R3FFESBQXPR7S6|          4|            0|          0|   N|
|R3QO6Z942CKH34|          5|            

# Load

In [21]:
mode = "append"
jdbc_url="jdbc:postgresql://hamdouna-pgdb-1.ctliexachjmo.ap-southeast-2.rds.amazonaws.com:5432/module_22_db"
config = {"user":"root", "password": "assignment", "driver":"org.postgresql.Driver"}

In [22]:
# Write review_id_df to table in RDS

review_id_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)


Py4JJavaError: ignored

In [18]:
# Write products_df to table in RDS

products_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)


Py4JJavaError: ignored

In [19]:
# Write customers_df to table in RDS

customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)


Py4JJavaError: ignored

In [20]:
# Write vine_df to table in RDS

vine_df.write.jdbc(url=jdbc_url, table="vine", mode=mode, properties=config)


Py4JJavaError: ignored